# DỰ ĐOÁN GIÁ SÁCH

# Giới thiệu
Bài toán dự đoán là một bài toán điển hình và thông qua đó, người ta có thể dự đoán được những kết quả mà ta quan tâm dựa trên những dữ liệu đã được khảo sát. Hôm nay, nhóm chúng em sẽ trình bày về cách crawl data các thông tin cần thiết khi dự đoán giá sách

# Nguồn tài liệu
`https://tiki.vn/`

Thêm thư viện

In [ ]:
from selenium import webdriver
from time import sleep
import pandas as pd

Cài đặt webdriver và trình duyệt Chrome

`https://sites.google.com/a/chromium.org/chromedriver/`

In [ ]:
# Thiet lap Wendriver cho trinh duyet Chrome
browser = webdriver.Chrome(executable_path="chromedriver.exe")

In [3]:
list_NXB = []
list_size = []
list_type_convert = []
list_num_page = []
list_title = []
list_sub_cate = []
list_cate = []
list_auth = []
list_seller_name = []
list_current_price = []
list_origin_price = []
list_rv_rating = []
list_sales = []

Truy cập vào đường link muốn crawl

In [4]:
url = "https://tiki.vn/sach-van-hoa-dia-ly-du-lich/c857?page=1"
browser.get(url)

Lấy tất cả những item sách hiện có trên trang web

In [ ]:
books = browser.find_elements_by_class_name('product-item')

arr_books = []
for book in books:
    link = book.get_attribute('href')       # Lay thuoc tinh href cua item sach
    arr_books.append(link)

Khởi tạo biến book lưu giá trị của sách

In [30]:
book = {
    'title': '',
    'sub-category': '',
    'category': '',
    'author': '',
    'seller-name': '',
    'current-price': '',
    'origin-price': '',
    'review-rating': '',
    'sales': '',
    'NXB': '',
    'size': '',
    'type_convert': '',
    'num_page': ''
}

Lặp qua các url để truy cập vào trang web muốn crawl dữ liệu

In [ ]:
for i in arr_books:
    browser.get(i)      # Truy cap vao tung duong link sach trong mang arr_books
    try:
        cates = browser.find_elements_by_class_name('breadcrumb-item')      # Lay cac category cua sach
        cates = [str(i.text) for i in cates]

        details = browser.find_elements_by_class_name('group')      # Lay thong tin chi tiet
        details = [str(i.text) for i in details][0].split('\n')
        
        show_link = browser.find_element_by_class_name('number')    # click de xem danh gia
        show_link.click()

        for i in details:
            if "Nhà xuất bản" in i:
                book['NXB'] = i.replace("Nhà xuất bản", "")
                continue

            if "Công ty phát hành" in i:
                book['NXB'] = i
                continue

            if "Kích thước" in i:
                book['size'] = i.replace("Kích thước", "")
                continue

            if "Loại bìa" in i:
                book['type_convert'] = i.replace("Loại bìa", "")
                continue

            if "Số trang" in i:
                book['num_page'] = i.replace("Số trang", "")
                continue

        book['title'] = browser.find_element_by_class_name("title").text    # Tra ve vao thuoc tinh title
        book['sub-category'] = cates[len(cates) - 2]
        book['category'] = cates[len(cates) - 3]
        book['author'] = browser.find_element_by_class_name('brand-and-author').text[9:]  # Tra ve thuoc tinh tac gia
        book['seller-name'] = browser.find_element_by_class_name('seller-name').text    # Tra ve thuoc tinh ten nguoi ban
        book['origin-price'] = browser.find_element_by_class_name("product-price__current-price").text # Tra ve thuoc tinh gia goc
        book['current-price'] = browser.find_element_by_class_name('product-price__list-price').text   # Tra ve thuoc tinh gia duoc sale
        book['review-rating'] = browser.find_element_by_class_name('review-rating__point').text     # Tra ve thuoc tinh luot danh gia
        book['sales'] = browser.find_element_by_class_name('below-title').text.split('\n')[1][7:] # Tra ve thuoc tinh so luot ban

    except Exception as e:
        print(e)
        continue
    
    # Gan vao mang cac gia tri cua thuoc tinh de chen vao excel
    list_title.append(book['title'])
    list_sub_cate.append(book['sub-category'])
    list_cate.append(book['category'])
    list_auth.append(book['author'])
    list_seller_name.append(book['seller-name'])
    list_current_price.append(book['current-price'])
    list_origin_price.append(book['origin-price'])
    list_rv_rating.append(book['review-rating'])
    list_sales.append(book['sales'])
    list_NXB.append(book['NXB'])
    list_size.append(book['size'])
    list_type_convert.append(book['type_convert'])
    list_num_page.append(book['num_page'])
    break

In [32]:
print(book)

{'title': 'Sách - Lão Tử Đạo Đức kinh - Nguyễn Hiến Lê ( Tuyển Tập Bách Gia Tranh Minh)', 'sub-category': 'Sách Danh Nhân', 'category': 'Sách Văn Hóa - Địa Lý - Du Lịch', 'author': 'Nguyễn Hiến Lê', 'seller-name': 'NVP', 'current-price': '159.000 ₫', 'origin-price': '99.000 ₫', 'review-rating': '4.8', 'sales': '24', 'NXB': ' Nhà Xuất Bản Hồng Đức', 'size': '', 'type_convert': ' Bìa mềm', 'num_page': ' 296'}


In [35]:
# Doc cac gia tri vao cac cot trong bang excel
data = pd.DataFrame({
    'title': list_title,
    'sub-category': list_sub_cate,
    'category': list_cate,
    'author': list_auth,
    'seller-name': list_seller_name,
    'current-price': list_current_price,
    'origin-price': list_origin_price,
    'review-rating': list_rv_rating,
    'sales': list_sales,
    'NXB': list_NXB,
    'size': list_size,
    'type_convert': list_type_convert,
    'num_page': list_num_page
})
data.to_excel('data.xlsx', sheet_name='sheet1', index=False)

In [41]:
import pandas as pd

df_data = pd.read_excel('data.xlsx')
print(df_data)

                                               title    sub-category  \
0  Sách - Lão Tử Đạo Đức kinh - Nguyễn Hiến Lê ( ...  Sách Danh Nhân   

                          category          author seller-name current-price  \
0  Sách Văn Hóa - Địa Lý - Du Lịch  Nguyễn Hiến Lê         NVP     159.000 ₫   

  origin-price  review-rating  sales                     NXB  size  \
0     99.000 ₫            4.8     24   Nhà Xuất Bản Hồng Đức   NaN   

  type_convert  num_page  
0      Bìa mềm       296  


Dong trinh duyet

In [23]:
browser.close()